### Importing and mapping netCDF data with xarray and cartopy

- Read data from a netCDF file with xarray
- Select (index) and modify variables using xarray
- Set up map features with cartopy (lat/lon tickmarks, continents, country/state borders, etc.)
- Overlay various plot types: contour lines, filled contours, vectors, and barbs
- Customize plot elements such as the colorbar and titles
- Save figure
    

In [ ]:
## Imports

import os, sys
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter


### Load netcdf data with Xarray

In this example, we show how to import and map ERA5 reanalysis data for an AR-Thunderstorm event that occurred in Santa Barbara County on 6 March 2019. We will use the `era5.AR-LTG.6hr.nc` sample data file, which contains 6-hourly ERA5 Reanalysis on a 0.5 x 0.5 deg lat-lon grid during 4-8 March (5 days). ERA5 data was retrieved from the Climate Data Store and subset to a regional domain over the Western US/N. Pacific. 

The xarray package provides an easy interface for importing and analyzing multidimensional data. Because xarray was designed around the netCDF data model, it is an especially useful tool for working with weather and climate data.


Xarray has two fundamental **data structures**:  

**1.** a **`DataArray`**, which holds a single n-dimensional variable. Elements of a DataArray include:
   - `values`: numpy array of data values
   - `dims`: list of named dimensions (for example, `['time','lat','lon']`)
   - `coords`: coordinate arrays (e.g., vectors of lat/lon values or datetime data)
   - `atts`: variable attributes such as `units` and `standard_name`   
    
**2.** a **`Dataset`**, which holds multiple n-dimensional variables (shared coordinates). Elements of a Dataset: data variables, dimensions, coordinates, and attributes.


In the cell below, we will load the ERA5 data (netcdf file) into an xarray dataset.


In [ ]:
# Path to ERA5 data
filepath = "../sample-data/era5.6hr.AR-thunderstorm.20190304_08.nc"

# Read nc file into xarray dataset
ds = xr.open_dataset(filepath)

# Print dataset contents 
print(ds)


### Selecting/Indexing data with xarray

We can always use regular numpy indexing and slicing on DataArrays and Datasets; however, 
it is often more powerful and easier to use xarray’s `.sel()` method of label-based indexing.


In [ ]:
# Select a single day 
ds.sel(time='2019-03-06')


In [ ]:
# Select range of lats (30-40 N)
# because ERA5 data latitudes are listed from 90N to 90S
# you have to slice from latmax to latmin

latmin=30
latmax=40
ds.sel(latitude=slice(latmax,latmin))


In [ ]:
# Select the date/time of the AR event (06 March 2019 at 06 UTC)



In [ ]:
# Subset data to a single pressure level: 250-hPa



In the following code block, we select the data and coordinate variables needed to create a map of 250-hPa heights and winds at the time of the AR-Thunderstorm event.

In [ ]:
# coordinate arrays
lats = dsAR['latitude'].values     # .values extracts var as numpy array
lons = dsAR['longitude'].values   
print(lats.shape, lons.shape)

# data variables
uwnd = dsAR['u'].values
#vwnd = 
#hgts = 

# check shape of new arrays and print some values...



### Do some calculations with our data...

Calculate the magnitude of the wind (wind speed) from u and v components. Then, convert wind speed from m/s to knots. 

In [ ]:
# Define a function to compute wind speed from u and v

def calc_wspd(u, v):
    """Computes wind speed from u and v components"""        
    
    wspd = np.sqrt(u**2 + v**2) 
    
    return wspd


In [ ]:
# Calculate wind speed using function

wspd = calc_wspd(uwnd, vwnd)
print(wspd.shape)


In [ ]:
# Define a function to convert wind from m/s to knots

# Hint: 1 m/s = 1.9438445 knots




In [ ]:
# Convert wspd to knots



### Plotting with Cartopy

In [ ]:
# Projection/Coordinate systems
datacrs = ccrs.PlateCarree()     # data/source
mapcrs = ccrs.PlateCarree()      # map/destination

# Map extent
lonmin = lons.min()
lonmax = lons.max()
latmin = lats.min()
latmax = lats.max()

# Tickmark Locations
# dx = np.arange(lonmin,lonmax+1,10)
# dy = np.arange(latmin,latmax+1,10)
# print(dx)
# print(dy)


In [ ]:
# Check range of data values (helps set up contour levels)

print(hgts.min(), hgts.max())
print(wspd_knots.min(), wspd_knots.max())


Create a basemap plot.

In [ ]:
# Create figure
fig = plt.figure(figsize=(10,7)) 

# Add plot axes and draw basemap
ax = fig.add_subplot(111, projection=mapcrs)
ax.set_extent([lonmin,lonmax,latmin,latmax], crs=mapcrs)

# xticks
ax.set_xticks(dx, crs=mapcrs)      
lon_formatter = LongitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
# yticks
ax.set_yticks(dy, crs=mapcrs)
lat_formatter = LatitudeFormatter()
ax.yaxis.set_major_formatter(lat_formatter)
# tick params
ax.tick_params(direction='out', labelsize=8.5, length=5, pad=2, color='black')    

# Map features
ax.add_feature(cfeature.LAND, facecolor='0.9') 
ax.add_feature(cfeature.BORDERS, edgecolor='0.1', linewidth=0.7)
ax.add_feature(cfeature.COASTLINE, edgecolor='k', linewidth=1.0)
ax.add_feature(cfeature.STATES, edgecolor='0.1', linewidth=0.7)

# show basemap
plt.show()

# Geopotenital height lines
# clevs_hgts = np.arange(840,1280,12)
# cs = ax.contour(lons, lats, hgts/10., transform=datacrs,
#                 levels=clevs_hgts,
#                 colors='blue',         # line color
#                 linewidths=1.1)     # line thickness
#                 
# # Add labels to contour lines
# plt.clabel(cs, fmt='%d',fontsize=8.5, inline_spacing=5)

# # Show
# plt.show()  


Make the draw a basemap a function.

In [ ]:
def draw_basemap():

    # Create figure
    fig = plt.figure(figsize=(10,7)) 

    # Add plot axes and draw basemap
    ax = fig.add_subplot(111, projection=mapcrs)
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=mapcrs)

    # xticks
    ax.set_xticks(dx, crs=mapcrs)      
    lon_formatter = LongitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    # yticks
    ax.set_yticks(dy, crs=mapcrs)
    lat_formatter = LatitudeFormatter()
    ax.yaxis.set_major_formatter(lat_formatter)
    # tick params
    ax.tick_params(direction='out', labelsize=8.5, length=5, pad=2, color='black')    

    # Map features
    ax.add_feature(cfeature.LAND, facecolor='0.9') 
    ax.add_feature(cfeature.BORDERS, edgecolor='0.1', linewidth=0.7)
    ax.add_feature(cfeature.COASTLINE, edgecolor='k', linewidth=1.0)
    ax.add_feature(cfeature.STATES, edgecolor='0.1', linewidth=0.7)
    
    return fig, ax


Using your basemap function, add contour lines and filled contours.

In [ ]:
# Draw basemap
fig, ax = draw_basemap()

# Geopotenital height lines
clevs_hgts = np.arange(840,1280,12)
cs = ax.contour(lons, lats, hgts/10., transform=datacrs,
                levels=clevs_hgts,
                colors='k',         # line color
                linewidths=1.2)     # line thickness
               
# Add labels to contour lines
plt.clabel(cs, fmt='%d',fontsize=8.5, inline_spacing=5)

# Wind speed - contour fill
clevs_wspd = np.arange(60,121,10)
cf = ax.contourf(lons, lats, wspd_knots, transform=datacrs,
                 levels=clevs_wspd, 
                 cmap='BuPu', 
                 extend='max',
                 alpha=0.9)   # transparency (0=transparent, 1=opaque)

# show
plt.show()


Add wind vectors using quiver. 

In [ ]:
# Draw basemap
fig, ax = draw_basemap()

# Geopotenital height lines
clevs_hgts = np.arange(840,1280,12)
cs = ax.contour(lons, lats, hgts/10., transform=datacrs,
                levels=clevs_hgts,
                colors='k',         # line color
                linewidths=1.2)     # line thickness
               
# Add labels to contour lines
plt.clabel(cs, fmt='%d',fontsize=8.5, inline_spacing=5)

# Wind speed - contour fill
clevs_wspd = np.arange(70,131,10)
cf = ax.contourf(lons, lats, wspd_knots, transform=datacrs,
                 levels=clevs_wspd, 
                 cmap='BuPu', 
                 extend='max',
                 alpha=0.8)   # transparency (0=transparent, 1=opaque)

# Wind vectors 
ax.quiver(lons, lats, uwnd, vwnd, transform=datacrs, 
          color='k', 
          regrid_shape=18, 
          pivot='middle')

# Plot title
titlestring = f"{plev}-hPa Hgts/Wind"
ax.set_title(titlestring, loc='left',fontsize=12)

# show
plt.show()


Plot wind barbs instead of vectors

In [ ]:
# Draw basemap
fig, ax = draw_basemap()

# Geopotenital height lines
clevs_hgts = np.arange(840,1280,12)
cs = ax.contour(lons, lats, hgts/10., transform=datacrs,
                levels=clevs_hgts,
                colors='b',         # line color
                linewidths=1.2)     # line thickness
               
# Add labels to contour lines
plt.clabel(cs, fmt='%d',fontsize=8.5, inline_spacing=5)

# Wind speed - contour fill
clevs_wspd = np.arange(70,131,10)
cf = ax.contourf(lons, lats, wspd_knots, transform=datacrs,
                 levels=clevs_wspd, 
                 cmap='BuPu', 
                 extend='max',
                 alpha=0.8)   # transparency (0=transparent, 1=opaque)

# Wind barbs
ax.barbs(lons, lats, uwnd, vwnd, transform=datacrs, 
          color='k', regrid_shape=12, pivot='middle')

# Plot title
titlestring = f"{plev}-hPa Hgts/Wind"
ax.set_title(titlestring, loc='left',fontsize=12)

# show
plt.show()
